<a href="https://colab.research.google.com/github/nihald2000/Edgeai/blob/main/LangChainExpressionLanguage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install -q \
    langchain \
    faiss-cpu \
    langchain-community \
    langchain-google-genai \
    bs4 \
    langchain-text-splitters \
    langchain_groq \
    google-ai-generativelanguage==0.6.15

In [21]:
### GOOGLE_API_KEY and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import google
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
if GROQ_API_KEY:
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY
    print(" GROQ_API_KEY loaded.")
else:
    print("GROQ_API_KEY not found in Colab Secrets. LangSmith tracing will not work.")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print(" GOOGLE_API_KEY API Key loaded.")
else:
    print("GOOGLE_API_KEY not found in Colab Secrets. LangSmith tracing will not work.")

LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
if LANGCHAIN_API_KEY:
    os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
    print(" LangSmith API Key loaded.")
else:
    print("LANGCHAIN_API_KEY not found in Colab Secrets. LangSmith tracing will not work.")


 GROQ_API_KEY loaded.
 GOOGLE_API_KEY API Key loaded.
 LangSmith API Key loaded.


In [22]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=GROQ_API_KEY)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fc847a3ba50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fc847a39150>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [23]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)
result

AIMessage(content='Here are a couple of ways to say "Hello, How are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?**  (pronounced: bohn-zhoor, kom-mahn ah-lay-voo)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (pronounced: sa-loo, kom-mahn vah-too)\n\n\nLet me know if you\'d like to know how to say something else in French!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 21, 'total_tokens': 127, 'completion_time': 0.192727273, 'prompt_time': 0.002130147, 'queue_time': 0.107741052, 'total_time': 0.19485742}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c020f57f-4528-4118-8c47-7d9347768110-0', usage_metadata={'input_tokens': 21, 'output_tokens': 106, 'total_tokens': 127})

In [24]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a couple of ways to say "Hello, How are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?**  (pronounced: bohn-zhoor, kom-mahn ah-lay-voo)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (pronounced: sa-loo, kom-mahn vah-too)\n\n\nLet me know if you\'d like to know how to say something else in French!\n'

In [25]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n'

In [26]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [27]:
result=prompt.invoke({"language":"French","text":"Hello"})
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [28]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'